# **Import Library**

In [ ]:
import os
from nltk import word_tokenize
from nltk import edit_distance
import itertools
import pandas as pd

In [ ]:
import nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# **Example of Levenshtein**

In [ ]:
a = input('String of A: ')
b = input('String of B: ')

String of A: lucky
String of B: lucy


In [ ]:
edit_distance(a, b)

1

In [ ]:
# edit a to b
matrix = []

# initialize
for i in range(len(a)+1):
    matrix.append([-1]*(len(b)+1))

matrix[0][0] = 0

for i in range(1,len(a)+1):
    matrix[i][0] = matrix[i-1][0] + 1

for i in range(1, len(b)+1):
    matrix[0][i] = matrix[0][i-1] + 1


for i in range(1, len(a)+1):
    for j in range(1, len(b)+1):
        if(b[j-1] == a[i-1]):
            matrix[i][j] = min(matrix[i-1][j-1], matrix[i-1][j]+1, matrix[i][j-1]+1)
        else:
            matrix[i][j] = min(matrix[i-1][j-1]+1, matrix[i-1][j]+1, matrix[i][j-1]+1)

matrix

[[0, 1, 2, 3, 4],
 [1, 0, 1, 2, 3],
 [2, 1, 0, 1, 2],
 [3, 2, 1, 0, 1],
 [4, 3, 2, 1, 1],
 [5, 4, 3, 2, 1]]

# **Implementation of Levensthein**

## Import File

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/levenshtein_try/dataset.csv')
dataset

,Number,Category,Action_needed,Question,Subcategory,Action,Time,Sentence
0,1,lights,1,0,kitchen,on,today,Illuminate the kitchen today.
1,2,lights,1,0,kitchen,on,tomorrow,Illuminate the kitchen tomorrow.
2,3,lights,1,0,kitchen,on,hour,Turn on the light in the kitchen in 10 hours.
3,4,lights,1,0,kitchen,on,day,Turn on the light in the kitchen in 1 day.
4,5,lights,1,0,diningroom,on,today,Illuminate the dining room today.
...,...,...,...,...,...,...,...,...
658,659,time,0,0,none,none,now,Time.
659,660,time,0,0,none,none,now,I don't want you to tell me the time.
660,661,time,1,0,none,none,hour,Tell me the time in an hour.
661,662,time,1,0,none,none,day,Tell me the time in a day.


In [ ]:
# define the sentence coloumn as independent var
sentences_df = dataset[['Sentence']]
sentences_df

,Sentence
0,Illuminate the kitchen today.
1,Illuminate the kitchen tomorrow.
2,Turn on the light in the kitchen in 10 hours.
3,Turn on the light in the kitchen in 1 day.
4,Illuminate the dining room today.
...,...
658,Time.
659,I don't want you to tell me the time.
660,Tell me the time in an hour.
661,Tell me the time in a day.


## Levenshtein Processing Using Toknizing for Spelling Correction

In [ ]:
# tokenize vocab from sentence
def get_plain_vocabluary():
    sentencess = [word_tokenize(sentence['Sentence']) for index, sentence in sentences_df.iterrows()]
    mergesentences = list(itertools.chain.from_iterable(sentencess))
    plainvocabulary = list(set(mergesentences))
    return plainvocabulary

In [ ]:
# levenshtein def
def levenshtein_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [ ]:
# spelling correction def
def spelling_correction(sentence):
    splittedsentence = word_tokenize(sentence)
    vocwords = list(itertools.chain.from_iterable([get_plain_vocabluary()]))
    for i,word in enumerate(splittedsentence):
        if (word not in vocwords and not word.isdigit()): # ignore digits
            levdistances = []
            for vocword in vocwords:
                levdistances.append(levenshtein_distance(word,vocword))
            splittedsentence[i] = vocwords[levdistances.index(min(levdistances))]
        else:
            splittedsentence[i] = word
    return ' '.join(splittedsentence)

## Testing

In [ ]:
sentence = input('Enter the Smart Home Command: ')

Enter the Smart Home Command: crose the dook


In [ ]:
print(spelling_correction(f'You mean, {sentence} ?'))

You mean 2 Close the door ?
